## Wide Residual Net

 - Training Dataset:  CutMix, beta=1, cutmix_prob=1
 - Sagemaker Notebook must be of type, conda_pytorch_p36
 
#### Install Requirements

In [5]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

In [6]:
!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


#### Train the Model

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    train.batch_size 128 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_CM_1 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00 \
    scheduler.epochs 400

[2020-07-10 23:30:35] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_CM_1
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
    alp

#### Refine the Model

In [ ]:
## Model hung during last 100 epochs -> Finish training from last checkpoint with augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00/checkpoint_00300.pth \
    dataset.name CIFAR10_CM_1 \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    train.batch_size 128 \
    train.base_lr 0.0008 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume300 \
    scheduler.epochs 100

#### Set LEARNING RATE based on ending LR

[2020-07-13 00:23:12] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_CM_1
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
    alp

In [7]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume300/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume300/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    train.batch_size 128 \
    train.base_lr 0.00016 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR

[2020-07-13 12:25:23] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: 

#### Evaluate the Model

In [8]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume400_50/test_results_0050_cifar10

[2020-07-13 18:49:36] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:27<00:00,  2.92it/s]
[2020-07-13 18:50:04] __main__ INFO: Elapsed 27.03
[2020-07-13 18:50:04] __main__ INFO: Loss 0.2518 Accuracy 0.9520


In [9]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume400_50/test_results_0050_cifar101

[2020-07-13 18:50:15] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:05<00:00,  2.82it/s]
[2020-07-13 18:50:21] __main__ INFO: Elapsed 5.69
[2020-07-13 18:50:21] __main__ INFO: Loss 0.6063 Accuracy 0.8830


In [11]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume300/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00/test_results_0400_cifar10

[2020-07-13 18:51:40] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume300/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:27<00:00,  2.92it/s]
[2020-07-13 18:52:09] __main__ INFO: Elapsed 27.07
[2020-07-13 18:52:09] __main__ INFO: Loss 0.2854 Accuracy 0.9323


In [12]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/wrn.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume300/checkpoint_00400.pth \
   dataset.name CIFAR101 \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00/test_results_0400_cifar101

[2020-07-13 18:52:28] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1/exp00_resume300/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:05<00:00,  2.80it/s]
[2020-07-13 18:52:34] __main__ INFO: Elapsed 5.71
[2020-07-13 18:52:34] __main__ INFO: Loss 0.5985 Accuracy 0.8560


#### Record the Results

In [3]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['wrn_28_10', 'wrn_28_10', 'wrn_28_10'],
           'Testset': ['cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 200, 200],
           'Loss': [0.2299, 0.1760, 0.3896],
           'Accuracy': [0.9311, 0.9578, 0.8975],
           'Original_Accuracy': [95.9, 95.9, 89.7],
           'Original_CI': [(95.5, 96.3), (95.5, 96.3), (88.3, 91.0)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])

#df.to_csv('/home/ec2-user/SageMaker/experiments/wrn_28_10/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,wrn_28_10,cifar10,100,0.2299,0.9311,95.9,"(95.5, 96.3)"
1,wrn_28_10,cifar10,200,0.1760,0.9578,95.9,"(95.5, 96.3)"
2,wrn_28_10,cifar10.1,200,0.3896,0.8975,89.7,"(88.3, 91.0)"


In [13]:
import pandas as pd
model = 'wrn_28_10_cm_1_1'
model_refined = model + '_refined400'

a = pd.Series([model, 400, 'cifar10', 0.2854, 0.9323])
c = pd.Series([model, 400, 'cifar10.1', 0.5985, 0.8560])

e = pd.Series([model_refined, 50, 'cifar10.1', 0.6063, 0.8830])
f = pd.Series([model_refined, 50, 'cifar10', 0.2518, 0.9520])
               
df_results = pd.concat([a,c,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 95.9 if row[2] == 'cifar10' else 89.7), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (95.5, 96.3) if row[2] == 'cifar10' else (88.3, 91.0)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/' + model + '/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,wrn_28_10_cm_1_1,400,cifar10,0.2854,0.9323,95.9,"(95.5, 96.3)"
1,wrn_28_10_cm_1_1,400,cifar10.1,0.5985,0.856,89.7,"(88.3, 91.0)"
2,wrn_28_10_cm_1_1_refined400,50,cifar10.1,0.6063,0.883,89.7,"(88.3, 91.0)"
3,wrn_28_10_cm_1_1_refined400,50,cifar10,0.2518,0.952,95.9,"(95.5, 96.3)"


In [15]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-june29'
prefix = 'sagemaker/results/original-models/wrn_28_10_cm_1_1'
path = '/home/ec2-user/SageMaker/experiments/wrn_28_10_cm_1_1'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)